## Trabajo Practico n° 2 - Organización de datos

## *Machine Learning*

### *Alumno: Sebastian Leonardi*

### *Año 2018 - 1er cuatrimestre*

In [129]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing

### Se cargan todos los Dataframes

In [2]:
postulantes_ed = pd.read_csv('./datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')
postulantes_ed_h15a = pd.read_csv('fiuba_desde_15_abril/fiuba_1_postulantes_educacion.csv')
postulantes_ed_d15a = pd.read_csv('fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv')
# avisos_online = pd.read_csv('datos_navent_fiuba/fiuba_5_avisos_online.csv')
# avisos_online_h15a = pd.read_csv('fiuba_hasta_15_abril/fiuba_5_avisos_online.csv')

In [3]:
postulantes_ge = pd.read_csv('datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv', parse_dates=['fechanacimiento'], date_parser=lambda d: pd.to_datetime(d, format="%Y/%m/%d", errors="coerce"))
postulantes_ge_h15a = pd.read_csv('fiuba_desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv', parse_dates=['fechanacimiento'], date_parser=lambda d: pd.to_datetime(d, format="%Y/%m/%d", errors="coerce"))
postulantes_ge_d15a = pd.read_csv('fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv', parse_dates=['fechanacimiento'], date_parser=lambda d: pd.to_datetime(d, format="%Y/%m/%d", errors="coerce"))

In [4]:
vistas = pd.read_csv('datos_navent_fiuba/fiuba_3_vistas.csv')
#vistas_d15a = pd.read_csv('fiuba_desde_15_abril/fiuba_3_vistas.csv')
vistas_h15a = pd.read_csv('fiuba_hasta_15_abril/fiuba_3_vistas.csv')

In [5]:
postulaciones = pd.read_csv('datos_navent_fiuba/fiuba_4_postulaciones.csv', parse_dates=['fechapostulacion'], date_parser=lambda d: pd.to_datetime(d, format="%Y/%m/%d", errors="coerce"))
postulaciones_h15a = pd.read_csv('fiuba_hasta_15_abril/fiuba_4_postulaciones.csv', parse_dates=['fechapostulacion'], date_parser=lambda d: pd.to_datetime(d, format="%Y/%m/%d", errors="coerce"))

In [6]:
avisos_detalle = pd.read_csv('datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
avisos_detalle_d15a = pd.read_csv('fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv')
avisos_detalle_h15a = pd.read_csv('fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv')
avisos_detalle_missing = pd.read_csv('fiuba_6_avisos_detalle_missing_nivel_laboral.csv')

In [7]:
#postulantes = pd.merge(postulantes_ed_h15a, postulantes_ed_d15a, how='left', on='idpostulante')

In [8]:
postulantes = postulantes_ed.append([postulantes_ed_h15a, postulantes_ed_d15a])

In [9]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1103193 entries, 0 to 407452
Data columns (total 3 columns):
idpostulante    1103193 non-null object
nombre          1103193 non-null object
estado          1103193 non-null object
dtypes: object(3)
memory usage: 33.7+ MB


In [10]:
postulantes.head(15)

,idpostulante,nombre,estado
0,NdJl,Posgrado,En Curso
1,8BkL,Universitario,En Curso
2,1d2B,Universitario,En Curso
3,NPBx,Universitario,En Curso
4,NPBx,Master,En Curso
5,Ym2X,Otro,En Curso
6,ZKl5,Master,En Curso
7,EDD99,Terciario/Técnico,En Curso
8,Y043Z,Universitario,En Curso
9,6M3jr,Universitario,En Curso


### Se transforman los niveles de estudios en numericos (tome el metodo de Jose para que queden iguales, ya que yo lo habia ponderado distinto)

In [11]:
#postulantes['nombre']=postulantes['nombre'].astype('category')

In [12]:
col_rename={'nombre':'estudios'}
postulantes=postulantes.rename(columns=col_rename)

In [13]:
postulantes['educacion']=postulantes['estudios'].astype('object')+postulantes['estado'].astype('object')

In [14]:
postulantes['educacion'].replace({
        'SecundarioAbandonado': 0,
        'SecundarioEn Curso': 1,
        'SecundarioGraduado': 2,
        'OtroAbandonado': 3,
        'OtroEn Curso': 4,
        'OtroGraduado': 5,
        'Terciario/TécnicoAbandonado': 6,
        'Terciario/TécnicoEn Curso': 7,
        'Terciario/TécnicoGraduado': 8,
        'UniversitarioAbandonado': 9,
        'UniversitarioEn Curso': 10,
        'UniversitarioGraduado': 11,
        'PosgradoAbandonado': 12,
        'PosgradoEn Curso': 13,
        'PosgradoGraduado': 14,
        'MasterAbandonado': 15,
        'MasterEn Curso': 16,
        'MasterGraduado': 17,
        'DoctoradoAbandonado': 18,
        'DoctoradoEn Curso': 19,
        'DoctoradoGraduado': 20,
    }, inplace=True)

### Se conserva solo el valor del estudio mas alto

In [15]:
postulantes=postulantes.groupby('idpostulante')['educacion'].max().to_frame().reset_index().sort_values('idpostulante', ascending=False)

In [16]:
postulantes.sample(10)

,idpostulante,educacion
287119,ZDao25b,2
164436,JBXL1xW,10
368835,lDLKYe6,2
171685,JBrYMMJ,2
41402,3NzYq2q,10
38450,3NQG2VP,9
201429,NAQ1YJ,14
255700,W9WNVb2,2
207903,NrvRbL,11
312477,akjAv80,5


### Trabajo con el genero de los postulantes

In [17]:
postulantes_ge_h15a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281385 entries, 0 to 281384
Data columns (total 3 columns):
idpostulante       281385 non-null object
fechanacimiento    268464 non-null datetime64[ns]
sexo               281385 non-null object
dtypes: datetime64[ns](1), object(2)
memory usage: 6.4+ MB


In [18]:
postulantes_ge_d15a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297747 entries, 0 to 297746
Data columns (total 3 columns):
idpostulante       297747 non-null object
fechanacimiento    281105 non-null datetime64[ns]
sexo               297747 non-null object
dtypes: datetime64[ns](1), object(2)
memory usage: 6.8+ MB


In [19]:
postulantes_ge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200888 entries, 0 to 200887
Data columns (total 3 columns):
idpostulante       200888 non-null object
fechanacimiento    196131 non-null datetime64[ns]
sexo               200888 non-null object
dtypes: datetime64[ns](1), object(2)
memory usage: 4.6+ MB


#### Se unifican todos los postulantes en un solo dataframe, se calcula la edad

In [20]:
postulantes_ge = postulantes_ge.append([postulantes_ge_h15a, postulantes_ge_d15a])

In [21]:
postulantes_ge.drop_duplicates('idpostulante', inplace=True)

In [22]:
postulantes_ge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504407 entries, 0 to 297183
Data columns (total 3 columns):
idpostulante       504407 non-null object
fechanacimiento    478073 non-null datetime64[ns]
sexo               504407 non-null object
dtypes: datetime64[ns](1), object(2)
memory usage: 15.4+ MB


In [23]:
postulantes_ge['edad'] = np.floor((pd.datetime.today() - postulantes_ge['fechanacimiento']).dt.days / 365)

In [24]:
postulantes_ge.sample(15)

,idpostulante,fechanacimiento,sexo,edad
97073,8MZ4WZ3,1992-11-09,FEM,25.0
47564,0zrPQqr,1990-09-07,FEM,27.0
39108,EpPjvb,1977-12-23,MASC,40.0
106614,96ZEEVP,1987-08-18,FEM,30.0
41185,4rWW4ae,1987-01-21,FEM,31.0
94411,lDXjB2M,1980-10-11,MASC,37.0
104188,GNdd8Ob,1996-01-19,FEM,22.0
31019,ZDPBaKE,1996-11-11,MASC,21.0
126180,3NQmDoJ,1991-03-28,FEM,27.0
30206,NGxZwJ,1981-07-23,FEM,36.0


#### Se convierte a booleano el sexo

In [25]:
postulantes_ge['sexo'] = postulantes_ge['sexo'].replace({'FEM': 0, 'MASC': 1}).astype(bool)

In [26]:
 del postulantes_ge['fechanacimiento']

In [27]:
postulantes_ge.sample(10)

,idpostulante,sexo,edad
18050,N0lmwJ,False,32.0
96661,6rP6W1O,False,33.0
213383,ZDrmdQ8,True,21.0
104073,xk9Y1qK,False,53.0
57243,zv8NooD,False,30.0
219769,PmV994b,False,29.0
38001,bOZpXbE,False,38.0
129002,W9kkG9m,True,27.0
181472,GNZRNx9,True,47.0
153637,kPjNwda,True,23.0


### Realizco el merge de los dos DF de postulantes

In [28]:
postulantes = pd.merge(postulantes, postulantes_ge, how='left', on='idpostulante')

In [29]:
postulantes.sample(10)

,idpostulante,educacion,sexo,edad
4030,zvPXaPJ,2,False,23.0
71952,mzb3rQ6,2,False,26.0
140971,ak50Vr9,6,True,36.0
412205,3NL4zDJ,11,False,28.0
76930,lDXQdr3,11,False,23.0
415020,2zmj2G0,11,False,29.0
75607,lDw4a5G,10,False,23.0
44285,qevAwaN,5,True,33.0
361247,8A1p1j,10,False,28.0
5223,zv8WoBk,10,True,25.0


### Trabajo con los avisos

In [30]:
avisos_detalle_h15a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18299 entries, 0 to 18298
Data columns (total 11 columns):
idaviso                 18299 non-null int64
idpais                  18299 non-null int64
titulo                  18299 non-null object
descripcion             18299 non-null object
nombre_zona             18299 non-null object
ciudad                  64 non-null object
mapacalle               1282 non-null object
tipo_de_trabajo         18299 non-null object
nivel_laboral           18299 non-null object
nombre_area             18299 non-null object
denominacion_empresa    18293 non-null object
dtypes: int64(2), object(9)
memory usage: 1.5+ MB


In [31]:
avisos_detalle_d15a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13798 entries, 0 to 13797
Data columns (total 11 columns):
idaviso                 13798 non-null int64
idpais                  13798 non-null int64
titulo                  13798 non-null object
descripcion             13798 non-null object
nombre_zona             13798 non-null object
ciudad                  86 non-null object
mapacalle               1019 non-null object
tipo_de_trabajo         13798 non-null object
nivel_laboral           13798 non-null object
nombre_area             13798 non-null object
denominacion_empresa    13795 non-null object
dtypes: int64(2), object(9)
memory usage: 1.2+ MB


In [32]:
avisos_detalle_missing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338 entries, 0 to 337
Data columns (total 11 columns):
idaviso                 338 non-null int64
idpais                  338 non-null int64
titulo                  338 non-null object
descripcion             338 non-null object
nombre_zona             338 non-null object
ciudad                  28 non-null object
mapacalle               271 non-null object
tipo_de_trabajo         338 non-null object
nivel_laboral           3 non-null object
nombre_area             338 non-null object
denominacion_empresa    338 non-null object
dtypes: int64(2), object(9)
memory usage: 29.1+ KB


In [33]:
avisos = avisos_detalle.append([avisos_detalle_h15a, avisos_detalle_d15a, avisos_detalle_missing])

In [34]:
avisos.drop_duplicates('idaviso', inplace=True)

In [35]:
avisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25288 entries, 0 to 337
Data columns (total 11 columns):
idaviso                 25288 non-null int64
idpais                  25288 non-null int64
titulo                  25288 non-null object
descripcion             25288 non-null object
nombre_zona             25288 non-null object
ciudad                  160 non-null object
mapacalle               2072 non-null object
tipo_de_trabajo         25288 non-null object
nivel_laboral           24953 non-null object
nombre_area             25288 non-null object
denominacion_empresa    25281 non-null object
dtypes: int64(2), object(9)
memory usage: 2.3+ MB


#### Se borran los features con poca informacion o poco relevante

In [36]:
del avisos['idpais']

In [37]:
del avisos['mapacalle']

In [38]:
del avisos['ciudad']

### Remuevo tags HTML en la descripciones

In [39]:
avisos['descripcion'] = avisos['descripcion'].str.replace('<.*?>', ' ')

In [40]:
avisos['descripcion'] = avisos['descripcion'].str.replace('&.*?;', ' ')

In [41]:
avisos['descripcion'] = avisos['descripcion'].str.replace('\r', ' ')

In [42]:
avisos['descripcion'] = avisos['descripcion'].str.replace('\n', ' ')

In [43]:
avisos.sample(10)

,idaviso,titulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
3252,1112225463,Outsourcing Manager,For international TELCO company we are looki...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Telecomunicaciones,CERUTTI & ASOCIADOS CONSULTING
4073,1112292689,Responsable Coordinación Logística,Para Importante Operadora Multinacional busca...,Gran Buenos Aires,Full-time,Jefe / Supervisor / Responsable,Logística,TALENT RECRUITERS
2031,1112345187,Jefe de Repuestos- Concesionaria,"Nuestro c liente, importante concesionaria ...",Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ventas,RANDSTAD
4269,1112296360,Ingeniero Eléctrico,Importante Compañía Multinacional Incorporar...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Mantenimiento,ARRIVE RRHH
9832,1112496759,05 May - C# (.NET ) Chief Software Architect (...,Al aplicar a la vacante será redirigido a la ...,Gran Buenos Aires,Teletrabajo,Senior / Semi-Senior,Tecnologia / Sistemas,CrossOver
10,1001724563,Paramédico,Nos encontramos en un proceso de selección pa...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Salud,Hochschild Mining
6426,1112467397,Director Creativo,CARESTINO INCORPORA: DIRECTOR CREATIVO (ZO...,Gran Buenos Aires,Full-time,Jefe / Supervisor / Responsable,Diseño Multimedia,BIAMER SRL
13813,1112392476,CAJERO/A JR. SUC. SANTA FE,Orientamos nuestra búsqueda a estudiantes ava...,Gran Buenos Aires,Full-time,Junior,Tesorería,BBVA Francés
11398,1112203334,importante restaurant seleccionará camareros,Se requiere - Muy buena presencia - Muy bue...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Gastronomia,PIOLA
3300,1112226448,Analista de Planeamiento de Inventario ( Zona ...,En KaizenRH buscamos URGENTE Analista de Pla...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Abastecimiento,Kaizen Recursos Humanos


### Trabajo con vistas

In [44]:
vistas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 961897 entries, 0 to 961896
Data columns (total 3 columns):
idAviso         961897 non-null int64
timestamp       961897 non-null object
idpostulante    961897 non-null object
dtypes: int64(1), object(2)
memory usage: 22.0+ MB


In [45]:
vistas_h15a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5758686 entries, 0 to 5758685
Data columns (total 3 columns):
idAviso         int64
timestamp       object
idpostulante    object
dtypes: int64(1), object(2)
memory usage: 131.8+ MB


In [46]:
#vistas_d15a.info() -- Se removio

In [47]:
#vistas_d15a.sample(15)

### Se unen los DF de vistas
### se agrupan y cuentan las vistas por postulante/aviso, para saber cuantas veces fue visto el aviso por el postulante 

In [48]:
#vistas = vistas.append([vistas_h15a, vistas_d15a])
#Se sacan las vistas despues del 15 de abril, porque no estan las postulaciones despues de esa fecha

In [49]:
vistas = vistas.append([vistas_h15a])

In [50]:
vistas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6720583 entries, 0 to 5758685
Data columns (total 3 columns):
idAviso         int64
timestamp       object
idpostulante    object
dtypes: int64(1), object(2)
memory usage: 205.1+ MB


In [51]:
cuenta_vistas = vistas.groupby(['idpostulante','idAviso'])['timestamp'].count().to_frame().reset_index()

In [52]:
cuenta_vistas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3568761 entries, 0 to 3568760
Data columns (total 3 columns):
idpostulante    object
idAviso         int64
timestamp       int64
dtypes: int64(2), object(1)
memory usage: 81.7+ MB


In [53]:
col_rename={'idAviso':'idaviso','timestamp':'vistas', }
cuenta_vistas=cuenta_vistas.rename(columns=col_rename)

In [54]:
cuenta_vistas.sample(15)

,idpostulante,idaviso,vistas
1329761,JBe65PJ,1112416123,2
251306,2zm1e1V,1112411350,1
1379987,JBx11VO,1112457520,2
550520,6LdPkr,1112454807,2
2355348,ZZzJKZ,1112167719,1
953548,A3X0MeG,1112334791,1
2780041,eko9VBB,1112426585,1
1845588,QNLwRoN,1112320618,1
307360,3NdRNqa,1112390398,7
2142435,YBMKZX,1112441384,1


### Trabajo con postulaciones

In [55]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3401623 entries, 0 to 3401622
Data columns (total 3 columns):
idaviso             int64
idpostulante        object
fechapostulacion    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 77.9+ MB


In [56]:
postulaciones_h15a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4909641 entries, 0 to 4909640
Data columns (total 3 columns):
idaviso             int64
idpostulante        object
fechapostulacion    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 112.4+ MB


In [57]:
postulaciones = postulaciones.append([postulaciones_h15a])

In [58]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8311264 entries, 0 to 4909640
Data columns (total 3 columns):
idaviso             int64
idpostulante        object
fechapostulacion    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 253.6+ MB


In [59]:
postulaciones.drop_duplicates(['idaviso','idpostulante','fechapostulacion'], inplace=True)

### Se unen los dataframes para formar el set de entrenamiento
#### en este merge podemos tener info de la relacion entre vistas y postulaciones
#### ademas formamos el dataframe final basandonos en las vistas de cada postulante. Con esto vamos a tener casos positivos (postulaciones concretadas) y casos negativos (fueron vistos los avisos, pero no hubo postulacion)
#### con esto vamos a tener un universo de casos completos, sin necesidad de generar casos aleatorios para tener negativos

In [60]:
trainingset = pd.merge(cuenta_vistas, postulaciones, how='left', on=['idaviso','idpostulante'])

In [61]:
trainingset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3569019 entries, 0 to 3569018
Data columns (total 4 columns):
idpostulante        object
idaviso             int64
vistas              int64
fechapostulacion    datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 136.1+ MB


In [62]:
trainingset.loc[trainingset.fechapostulacion.notnull(), 'postulo']=1

In [63]:
trainingset.loc[trainingset.fechapostulacion.isnull(), 'postulo']=0

In [64]:
 del trainingset['fechapostulacion']

In [65]:
trainingset[trainingset.postulo==1].count()

idpostulante    1698904
idaviso         1698904
vistas          1698904
postulo         1698904
dtype: int64

In [66]:
trainingset[trainingset.postulo==0].count()

idpostulante    1870115
idaviso         1870115
vistas          1870115
postulo         1870115
dtype: int64

### Se agrega el dataframe de postulantes

In [67]:
trainingset = pd.merge(trainingset, postulantes, how='left', on='idpostulante')

In [68]:
trainingset[trainingset.educacion.notnull()].count() 

idpostulante    3403723
idaviso         3403723
vistas          3403723
postulo         3403723
educacion       3403723
sexo            3403723
edad            3348140
dtype: int64

In [69]:
trainingset[trainingset.educacion.isnull()].count()

idpostulante    165296
idaviso         165296
vistas          165296
postulo         165296
educacion            0
sexo                 0
edad                 0
dtype: int64

#### Relleno las edades nulas con el valor promedio de edad

In [70]:
trainingset.loc[trainingset.edad.isnull(), 'edad'] =np.round(trainingset[trainingset.edad.notnull()].edad.mean(), decimals=2 )

#### Se rellena la educacion nula con educacion promedio

In [71]:
#np.round(trainingset[trainingset.educacion.notnull()].educacion.mean(), decimals=2 )

In [72]:
#trainingset.loc[trainingset.educacion.isnull(), 'educacion'] = 8

#### Se rellena el sexo como indefinido

In [73]:
#trainingset['sexo'].interpolate(method='linear')

#### Elimino las vistas de postulantes que no existen

In [74]:
trainingset = trainingset[trainingset.educacion.notnull() ]

In [75]:
trainingset.sample(20)

,idpostulante,idaviso,vistas,postulo,educacion,sexo,edad
1939443,RzrY83E,1112390566,1,1.0,11.0,False,34.00
1173957,EzEVeGo,1112369030,1,0.0,8.0,True,29.00
2512831,akjzXv0,1112413716,1,0.0,8.0,True,24.00
3157659,pzdr54Z,1112410056,2,1.0,11.0,True,23.00
3348863,vVjbEMe,1112448287,1,0.0,10.0,True,29.37
715813,8MB34Jl,1112424076,2,1.0,2.0,True,26.00
617918,6rP3PGx,1112424876,1,0.0,10.0,False,19.00
583183,6r2Bapj,1112366936,1,0.0,10.0,True,27.00
2568894,bOVr3G9,1112334788,1,1.0,2.0,False,23.00
3225757,qew8m2x,1112366747,2,0.0,4.0,True,26.00


#### compruebo estudios y sexo nulo

In [76]:
trainingset[trainingset['sexo'].isnull()].count()

idpostulante    0
idaviso         0
vistas          0
postulo         0
educacion       0
sexo            0
edad            0
dtype: int64

### Agrego el dataframe de avisos

In [77]:
trainingset = pd.merge(trainingset, avisos, how='left', on='idaviso')

In [78]:
trainingset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3403723 entries, 0 to 3403722
Data columns (total 14 columns):
idpostulante            object
idaviso                 int64
vistas                  int64
postulo                 float64
educacion               float64
sexo                    object
edad                    float64
titulo                  object
descripcion             object
nombre_zona             object
tipo_de_trabajo         object
nivel_laboral           object
nombre_area             object
denominacion_empresa    object
dtypes: float64(3), int64(2), object(9)
memory usage: 389.5+ MB


####  Se cuentan los casos con toda la info completa y postulaciones positivas  o negativas

In [79]:
trainingset[trainingset['titulo'].notnull() & trainingset['educacion'].notnull() & trainingset['postulo']==1].count()

idpostulante            1630551
idaviso                 1630551
vistas                  1630551
postulo                 1630551
educacion               1630551
sexo                    1630551
edad                    1630551
titulo                  1630551
descripcion             1630551
nombre_zona             1630551
tipo_de_trabajo         1630551
nivel_laboral           1630551
nombre_area             1630551
denominacion_empresa    1630529
dtype: int64

In [80]:
trainingset[trainingset['titulo'].notnull() & (trainingset['educacion'].notnull()) & (trainingset['postulo']==0)].count()

idpostulante            1773172
idaviso                 1773172
vistas                  1773172
postulo                 1773172
educacion               1773172
sexo                    1773172
edad                    1773172
titulo                  1773172
descripcion             1773172
nombre_zona             1773172
tipo_de_trabajo         1773172
nivel_laboral           1773172
nombre_area             1773172
denominacion_empresa    1772995
dtype: int64

### Finalmente tenemos un dataset de casos con postulaciones positivas y negativas, formados en base a las vistas de los postulantes. 
#### Se listan anvisos que tienen todos los features completos - son casi 3,5 millones.

In [81]:
trainingset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3403723 entries, 0 to 3403722
Data columns (total 14 columns):
idpostulante            object
idaviso                 int64
vistas                  int64
postulo                 float64
educacion               float64
sexo                    object
edad                    float64
titulo                  object
descripcion             object
nombre_zona             object
tipo_de_trabajo         object
nivel_laboral           object
nombre_area             object
denominacion_empresa    object
dtypes: float64(3), int64(2), object(9)
memory usage: 389.5+ MB


In [82]:
trainingset.shape

(3403723, 14)

### Se guarda el dataset Base 

In [83]:
trainingset.to_pickle('full_trainingset')

#### Se pasan las categorias a one hot encoding

In [84]:
categorias = trainingset.select_dtypes(include=['object']).copy()

In [85]:
categorias.sample(3)

,idpostulante,sexo,titulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
1340906,KBalo6A,False,Representantes telefónicos para venta de asist...,Nuestro call center incorporará a su equipo r...,Gran Buenos Aires,Part-time,Senior / Semi-Senior,Telemarketing,Aseguratuviaje.com
1578566,Nwq5oJ,False,Personal para atención al cliente telefónico P...,"Empresa multinacional de Servicios, se encuen...",Gran Buenos Aires,Part-time,Senior / Semi-Senior,Atención al Cliente,RH Talentum
1366870,KBrkKkj,False,Personal para atención al cliente telefónico P...,"Empresa multinacional de Servicios, se encuen...",Gran Buenos Aires,Part-time,Senior / Semi-Senior,Atención al Cliente,RH Talentum


In [86]:
del categorias['titulo']
del categorias['descripcion']

In [87]:
categorias.shape

(3403723, 7)

In [88]:
categorias.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3403723 entries, 0 to 3403722
Data columns (total 7 columns):
idpostulante            object
sexo                    object
nombre_zona             object
tipo_de_trabajo         object
nivel_laboral           object
nombre_area             object
denominacion_empresa    object
dtypes: object(7)
memory usage: 207.7+ MB


#### Conversion One hot Encoding

In [89]:
#categorias_onehot = pd.get_dummies(categorias, columns=['nombre_zona','tipo_de_trabajo', 'nivel_laboral'], drop_first=True)

In [90]:
#categorias_onehot = pd.get_dummies(categorias, columns=['sexo'], drop_first=True)

In [91]:
#from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()

#### Codificando nombre_zona

In [92]:
#columns=['nombre_zona','tipo_de_trabajo', 'nivel_laboral', 'nombre_area']
categorias_onehot = lb.fit_transform(categorias['nombre_zona'])

In [93]:
categorias_tags = lb.classes_

In [94]:
categorias_onehot = pd.DataFrame(data=categorias_onehot, columns=categorias_tags)

In [95]:
categorias_onehot.shape

(3403723, 4)

#### Uno el DF con el trainingset

In [96]:
#nombre_zona = categorias_onehot

trainingset = pd.concat([trainingset, categorias_onehot], axis=1)

In [97]:
trainingset.head(3)

,idpostulante,idaviso,vistas,postulo,educacion,sexo,edad,titulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,Buenos Aires (fuera de GBA),Capital Federal,GBA Oeste,Gran Buenos Aires
0,0z5Dmrd,1112384041,1,1.0,11.0,True,52.0,Gerente de RRHH- Para Importante Cadena de Far...,Me encuentro en la busqueda de un Gerente de...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Recursos Humanos,PilarArtigas,0,0,0,1
1,0z5Dmrd,1112418961,1,0.0,11.0,True,52.0,Coordinador de RRHH para Importante Sanatorio/...,Para Importante Sanatorio de Córdoba Capital ...,Gran Buenos Aires,Full-time,Jefe / Supervisor / Responsable,Recursos Humanos,GMS S.A.,0,0,0,1
2,0z5Dmrd,1112420060,1,1.0,11.0,True,52.0,Coordinador/a de RRHH,Empresa Multinacional Se encuentra en la búsq...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Recursos Humanos,Importante Empresa,0,0,0,1


#### Codificando tipo_de_trabajo

In [98]:
#columns=['nombre_zona','tipo_de_trabajo', 'nivel_laboral', 'nombre_area']
categorias_onehot = lb.fit_transform(categorias['tipo_de_trabajo'])

In [99]:
categorias_tags = lb.classes_

In [100]:
categorias_onehot = pd.DataFrame(data=categorias_onehot, columns=categorias_tags)

In [101]:
categorias_onehot.shape

(3403723, 10)

In [102]:
trainingset = pd.concat([trainingset, categorias_onehot], axis=1)

In [103]:
trainingset.head(3)

,idpostulante,idaviso,vistas,postulo,educacion,sexo,edad,titulo,descripcion,nombre_zona,...,Fines de Semana,Full-time,Part-time,Pasantia,Por Contrato,Por Horas,Primer empleo,Teletrabajo,Temporario,Voluntario
0,0z5Dmrd,1112384041,1,1.0,11.0,True,52.0,Gerente de RRHH- Para Importante Cadena de Far...,Me encuentro en la busqueda de un Gerente de...,Gran Buenos Aires,...,0,1,0,0,0,0,0,0,0,0
1,0z5Dmrd,1112418961,1,0.0,11.0,True,52.0,Coordinador de RRHH para Importante Sanatorio/...,Para Importante Sanatorio de Córdoba Capital ...,Gran Buenos Aires,...,0,1,0,0,0,0,0,0,0,0
2,0z5Dmrd,1112420060,1,1.0,11.0,True,52.0,Coordinador/a de RRHH,Empresa Multinacional Se encuentra en la búsq...,Gran Buenos Aires,...,0,1,0,0,0,0,0,0,0,0


#### Codificando nivel_laboral

In [104]:
#columns=['nombre_zona','tipo_de_trabajo', 'nivel_laboral', 'nombre_area']
categorias_onehot = lb.fit_transform(categorias['nivel_laboral'])

In [105]:
categorias_tags = lb.classes_

In [106]:
categorias_onehot = pd.DataFrame(data=categorias_onehot, columns=categorias_tags)

In [107]:
categorias_onehot.shape

(3403723, 5)

In [108]:
trainingset = pd.concat([trainingset, categorias_onehot], axis=1)

In [109]:
trainingset.head(3)

,idpostulante,idaviso,vistas,postulo,educacion,sexo,edad,titulo,descripcion,nombre_zona,...,Por Horas,Primer empleo,Teletrabajo,Temporario,Voluntario,Gerencia / Alta Gerencia / Dirección,Jefe / Supervisor / Responsable,Junior,Otro,Senior / Semi-Senior
0,0z5Dmrd,1112384041,1,1.0,11.0,True,52.0,Gerente de RRHH- Para Importante Cadena de Far...,Me encuentro en la busqueda de un Gerente de...,Gran Buenos Aires,...,0,0,0,0,0,0,0,0,0,1
1,0z5Dmrd,1112418961,1,0.0,11.0,True,52.0,Coordinador de RRHH para Importante Sanatorio/...,Para Importante Sanatorio de Córdoba Capital ...,Gran Buenos Aires,...,0,0,0,0,0,0,1,0,0,0
2,0z5Dmrd,1112420060,1,1.0,11.0,True,52.0,Coordinador/a de RRHH,Empresa Multinacional Se encuentra en la búsq...,Gran Buenos Aires,...,0,0,0,0,0,0,0,0,0,1


#### Codificando nombre_area

In [110]:
#columns=['nombre_zona','tipo_de_trabajo', 'nivel_laboral', 'nombre_area']
categorias_onehot = lb.fit_transform(categorias['nombre_area'])

In [111]:
categorias_tags = lb.classes_

In [112]:
categorias_onehot = pd.DataFrame(data=categorias_onehot, columns=categorias_tags)

In [113]:
categorias_onehot.shape

(3403723, 183)

In [114]:
trainingset = pd.concat([trainingset, categorias_onehot], axis=1)

MemoryError: 

In [115]:
trainingset.head(3)

,idpostulante,idaviso,vistas,postulo,educacion,sexo,edad,titulo,descripcion,nombre_zona,...,Por Horas,Primer empleo,Teletrabajo,Temporario,Voluntario,Gerencia / Alta Gerencia / Dirección,Jefe / Supervisor / Responsable,Junior,Otro,Senior / Semi-Senior
0,0z5Dmrd,1112384041,1,1.0,11.0,True,52.0,Gerente de RRHH- Para Importante Cadena de Far...,Me encuentro en la busqueda de un Gerente de...,Gran Buenos Aires,...,0,0,0,0,0,0,0,0,0,1
1,0z5Dmrd,1112418961,1,0.0,11.0,True,52.0,Coordinador de RRHH para Importante Sanatorio/...,Para Importante Sanatorio de Córdoba Capital ...,Gran Buenos Aires,...,0,0,0,0,0,0,1,0,0,0
2,0z5Dmrd,1112420060,1,1.0,11.0,True,52.0,Coordinador/a de RRHH,Empresa Multinacional Se encuentra en la búsq...,Gran Buenos Aires,...,0,0,0,0,0,0,0,0,0,1


### Finalmente se borran las columnas viejas y se guardan

In [116]:
del trainingset['nombre_zona']

In [117]:
del trainingset['tipo_de_trabajo']

In [118]:
del trainingset['nivel_laboral']

In [119]:
del trainingset['nombre_area']

### los siguientes features podrian ser considerados a futuro realizando una limpieza de terminos y reduciendo las dimensiones

In [121]:
del trainingset['titulo']

In [122]:
del trainingset['descripcion']

In [126]:
del trainingset['denominacion_empresa']

In [127]:
trainingset.head(3)

,idpostulante,idaviso,vistas,postulo,educacion,sexo,edad,Buenos Aires (fuera de GBA),Capital Federal,GBA Oeste,...,Por Horas,Primer empleo,Teletrabajo,Temporario,Voluntario,Gerencia / Alta Gerencia / Dirección,Jefe / Supervisor / Responsable,Junior,Otro,Senior / Semi-Senior
0,0z5Dmrd,1112384041,1,1.0,11.0,True,52.0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0z5Dmrd,1112418961,1,0.0,11.0,True,52.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0z5Dmrd,1112420060,1,1.0,11.0,True,52.0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### Se normalizan las columnas edad y educacion

In [134]:
x = trainingset[['edad']].values

In [135]:
min_max_scaler = preprocessing.MinMaxScaler()

In [136]:
x_scaled = min_max_scaler.fit_transform(x)

In [139]:
trainingset['edad'] = x_scaled

#### Estudioss

In [141]:
x = trainingset[['educacion']].values

In [142]:
min_max_scaler = preprocessing.MinMaxScaler()

In [143]:
x_scaled = min_max_scaler.fit_transform(x)

In [145]:
trainingset['educacion'] = x_scaled

In [146]:
trainingset.head(10)

,idpostulante,idaviso,vistas,postulo,educacion,sexo,edad,Buenos Aires (fuera de GBA),Capital Federal,GBA Oeste,...,Por Horas,Primer empleo,Teletrabajo,Temporario,Voluntario,Gerencia / Alta Gerencia / Dirección,Jefe / Supervisor / Responsable,Junior,Otro,Senior / Semi-Senior
0,0z5Dmrd,1112384041,1,1.0,0.55,True,0.401869,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0z5Dmrd,1112418961,1,0.0,0.55,True,0.401869,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0z5Dmrd,1112420060,1,1.0,0.55,True,0.401869,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0z5Dmrd,1112426253,2,0.0,0.55,True,0.401869,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0z5JW1r,1112165954,2,0.0,0.45,True,0.355140,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,0z5JW1r,1112394797,1,1.0,0.45,True,0.355140,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,0z5JW1r,1112417455,3,1.0,0.45,True,0.355140,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,0z5JW1r,1112428361,2,1.0,0.45,True,0.355140,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8,0z5JW1r,1112450843,2,0.0,0.45,True,0.355140,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,0z5VvGv,1111413600,1,0.0,0.10,True,0.196262,0,1,0,...,0,0,0,0,0,0,0,0,1,0


### Se guarda el DATASET completo y final

In [147]:
trainingset.to_pickle('final_trainingset')